# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
from nltk.stem import WordNetLemmatizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline,FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split,GridSearchCV,RandomizedSearchCV
from sklearn.metrics import classification_report
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.neighbors import KNeighborsClassifier
import joblib

In [2]:
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger','stopwords'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
# load data from database
engine = create_engine('sqlite:///Emergency.db')
df = pd.read_sql_table('Messages', 'sqlite:///Emergency.db')
df.drop(['categories'],axis=1,inplace=True)
df['related']=df['related'].replace(2,1)
X = df.message
Y = df.iloc[:,4:]
target_names=Y.columns.tolist()

In [10]:
Y.related.unique()

array([1, 0])

### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):
    text=text.lower()
    text=re.sub(r"[^a-zA-Z0-9]", " ",text)
    words=word_tokenize(text)
    words=[w for w in words if w not in stopwords.words("english")]
    
    lemmatizer=WordNetLemmatizer()
    clean_tokens=[]
    
    for t in words:
        tokens=lemmatizer.lemmatize(t).strip()
        clean_tokens.append(tokens)
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
pipeline = Pipeline([
    ('count_vect',CountVectorizer(tokenizer=tokenize)),
    ('tfidf',TfidfTransformer()),
    ('clf',MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
X_train,X_test,y_train,y_test=train_test_split(X,Y)
pipeline.fit(X_train,y_train)

Pipeline(memory=None,
     steps=[('count_vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
       ...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [9]:
y_pred=pipeline.predict(X_test)

In [10]:
y_test.shape,y_pred.shape,Y.columns.values

((6554, 36),
 (6554, 36),
 array(['related', 'request', 'offer', 'aid_related', 'medical_help',
        'medical_products', 'search_and_rescue', 'security', 'military',
        'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
        'missing_people', 'refugees', 'death', 'other_aid',
        'infrastructure_related', 'transport', 'buildings', 'electricity',
        'tools', 'hospitals', 'shops', 'aid_centers',
        'other_infrastructure', 'weather_related', 'floods', 'storm',
        'fire', 'earthquake', 'cold', 'other_weather', 'direct_report'], dtype=object))

In [14]:
indx=-1
for column in target_names:
    indx+=1
    print(column)
    print("_"*50)
    print(classification_report(y_test[column],y_pred[:,indx]))

related
__________________________________________________
             precision    recall  f1-score   support

          0       0.62      0.45      0.52      1542
          1       0.84      0.92      0.88      5012

avg / total       0.79      0.81      0.80      6554

request
__________________________________________________
             precision    recall  f1-score   support

          0       0.90      0.98      0.94      5453
          1       0.81      0.44      0.57      1101

avg / total       0.88      0.89      0.87      6554

offer
__________________________________________________
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      6512
          1       0.00      0.00      0.00        42

avg / total       0.99      0.99      0.99      6554

aid_related
__________________________________________________
             precision    recall  f1-score   support

          0       0.75      0.85      0.80      3840
          

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [11]:
parameters = {
    'clf__estimator__n_estimators':[5,10],
    'clf__estimator__max_depth':[None,10]
    #'clf__estimator__min_samples_split':[2,5,10],
    }

cv = GridSearchCV(pipeline,param_grid=parameters,verbose=3)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [12]:
cv.fit(X_train,y_train)
y_pred=cv.predict(X_test)
print(classification_report(y_test,y_pred,target_names=Y.columns.values))
print("Best Params:",cv.best_params_)

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] clf__estimator__max_depth=None, clf__estimator__n_estimators=5 ..
[CV]  clf__estimator__max_depth=None, clf__estimator__n_estimators=5, score=0.20109856576136712, total= 1.5min
[CV] clf__estimator__max_depth=None, clf__estimator__n_estimators=5 ..


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  2.4min remaining:    0.0s


[CV]  clf__estimator__max_depth=None, clf__estimator__n_estimators=5, score=0.198504729935917, total= 1.5min
[CV] clf__estimator__max_depth=None, clf__estimator__n_estimators=5 ..


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  4.8min remaining:    0.0s


[CV]  clf__estimator__max_depth=None, clf__estimator__n_estimators=5, score=0.20643881599023498, total= 1.5min
[CV] clf__estimator__max_depth=None, clf__estimator__n_estimators=10 .
[CV]  clf__estimator__max_depth=None, clf__estimator__n_estimators=10, score=0.24214220323466584, total= 1.8min
[CV] clf__estimator__max_depth=None, clf__estimator__n_estimators=10 .
[CV]  clf__estimator__max_depth=None, clf__estimator__n_estimators=10, score=0.23588648153799208, total= 1.8min
[CV] clf__estimator__max_depth=None, clf__estimator__n_estimators=10 .
[CV]  clf__estimator__max_depth=None, clf__estimator__n_estimators=10, score=0.23832773878547453, total= 1.8min
[CV] clf__estimator__max_depth=10, clf__estimator__n_estimators=5 ....
[CV]  clf__estimator__max_depth=10, clf__estimator__n_estimators=5, score=0.18782422947818125, total= 1.3min
[CV] clf__estimator__max_depth=10, clf__estimator__n_estimators=5 ....
[CV]  clf__estimator__max_depth=10, clf__estimator__n_estimators=5, score=0.1943851083307

[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed: 28.3min finished


                        precision    recall  f1-score   support

               related       0.85      0.93      0.89      5050
               request       0.83      0.45      0.58      1128
                 offer       0.00      0.00      0.00        29
           aid_related       0.74      0.61      0.67      2702
          medical_help       0.52      0.09      0.15       513
      medical_products       0.72      0.13      0.22       319
     search_and_rescue       0.90      0.05      0.10       179
              security       0.00      0.00      0.00       117
              military       0.61      0.10      0.17       207
           child_alone       0.00      0.00      0.00         0
                 water       0.89      0.34      0.49       418
                  food       0.85      0.53      0.65       733
               shelter       0.84      0.32      0.46       602
              clothing       0.70      0.16      0.25        90
                 money       0.83      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [47]:
class TextLengthExtractor(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return [[len(text)] for text in X]

In [48]:
pipeline1 = Pipeline([
        ('features', FeatureUnion([
            
            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),

            ('text_ext', TextLengthExtractor())
        ])),
    
        ('clf', MultiOutputClassifier(KNeighborsClassifier()))
    ])

In [49]:
parameters1 = {
    'clf__estimator__n_neighbors':[3,5],
    'clf__estimator__leaf_size':[5,10]
    #'clf__estimator__min_samples_split':[2,5,10],
    }

cv1 = GridSearchCV(pipeline1,param_grid=parameters1,verbose=3)

In [ ]:
cv1.fit(X_train,y_train)
y_pred=cv1.predict(X_test)
print(classification_report(y_test,y_pred,target_names=Y.columns.values))
print("Best Params:",cv1.best_params_)

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] clf__estimator__leaf_size=5, clf__estimator__n_neighbors=3 ......
[CV]  clf__estimator__leaf_size=5, clf__estimator__n_neighbors=3, score=0.1484589563625267, total= 3.3min
[CV] clf__estimator__leaf_size=5, clf__estimator__n_neighbors=3 ......


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  8.2min remaining:    0.0s


[CV]  clf__estimator__leaf_size=5, clf__estimator__n_neighbors=3, score=0.1464754348489472, total= 3.4min
[CV] clf__estimator__leaf_size=5, clf__estimator__n_neighbors=3 ......


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed: 16.5min remaining:    0.0s


### 9. Export your model as a pickle file

In [ ]:
joblib.dump(pipeline,'model.pkl')

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.